# Setup

In [1]:
import pandas as pd
import numpy as np
import src.maude_interface as maude
import logging
import os

logging.basicConfig(level=logging.DEBUG)
logging.debug("Current working dir: " + str(os.getcwd()))
data_folder = "../../data/"
reference_folder = "../reference/"

DEBUG:root:Current working dir: D:\WindowsSpace\Extra Big Files\ENME695\maude-analysis\analysis


# Code used for Initial testing/development
## Compile Database to analyze (*deprecated* in favor of maude.compile_maude_database)

In [ ]:
mdr_base = maude.read_mdr_file(data_folder + "mdrfoi-test.txt")
mdr_base.info()
mdr_base.head()

In [ ]:
# Tests to prove merging data and making large database.
mdr_base = maude.maude_to_pandas(data_folder + "mdrfoi-test.txt")
#foidev_test = maude.maude_to_pandas("./data/foidev.txt")
mdr_total = maude.add_data_to_mdr(mdr_base, (data_folder + "foidev1998.txt", data_folder + "foidevproblem.txt"))
mdr_total = pd.merge(mdr_total, maude.maude_to_pandas(reference_folder + "deviceproblemcodes.txt"), how='left', on='DEVICE_PROBLEM_CODE')

## Save Database for later

In [ ]:
mdr_total.to_pickle(data_folder + "mdrfoi_manual_compiled.pkl")
print(mdr_total.shape)
print(mdr_total.columns)
#print(mdr_total)

## Check if a CSV File is properly formatted

In [ ]:
maude.check_bad_csv(data_folder + "excluded/foidev.txt")

In [ ]:
foidev_dtype = {'MDR_REPORT_KEY': np.float32, 'DEVICE_PROBLEM_CODE': np.float32}
foidev_df = maude.maude_to_pandas(data_folder + "foidevproblem.txt", np.float32)
#print(foidev_df.dtypes)
print(foidev_df.info())
#foidev_df.apply(pd.to_numeric, errors='coerce', downcast='float')
#print(foidev_df.info())
print(foidev_df.isnull().sum())

In [ ]:
del mdr_base
del mdr_total
del foidev_df

In [ ]:
# Tries casting a column to categories. Used for classification, like hardware/software issue
problem_codes = maude.maude_to_pandas(reference_folder + "deviceproblemcodes.txt")
problem_codes['ERR_TYPE'] = problem_codes['ERR_TYPE'].astype('category')
print(problem_codes.describe())
print("\n\nProblem Code categories:\n" + str(problem_codes['ERR_TYPE'].value_counts()))

# PRODUCTION CODE
## New Way to Generate ENTIRE MAUDE Database

In [2]:
# alternative to all above statements except pickle
all_data = maude.compile_maude_database(data_folder, reference_folder, "mdrfoiThru2017.txt")
all_data.to_pickle(data_folder + "all_total_data.pkl")

DEBUG:root:Constructing database from base MDR file: ../../data/mdrfoiThru2017.txt


KeyboardInterrupt: 

## Perform Learning/Analysis

In [ ]:
all_text_data = maude.get_all_text_data(data_folder)
all_text_data.info()
print(all_text_data.shape)

Save All Text Data

In [ ]:
all_text_data.to_pickle(data_folder + "all_text_data.pkl")

### Stem

In [ ]:
# preprocessing
just_text = all_text_data['FOI_TEXT'].dropna()
from textblob import TextBlob

def stem_blob(input):
    if input is not np.nan:
        return TextBlob(" ").join((w.stem() for w in TextBlob(input).words.lower()))
    else:
        return input

just_text.apply(stem_blob)

### Save Stemmed Text

In [ ]:
just_text.to_pickle(data_folder + "just_text_stemmed.pkl")